In [1]:
!pip install transformers datasets wandb huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

In [2]:
from huggingface_hub import notebook_login
notebook_login()

import wandb
wandb.login()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saisurajs2199 (saisurajs2199-northeastern-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
from datasets import Dataset

data = {
    'prompt': [
        "What is the perfect time to have breakfast?",
        "When should we eat fruits?",
        "Is it good to eat fruits at night?",
        "What is the ideal time for dinner?",
        "How much water should we drink daily?"
        "How much vegetables, fruits and carbohydrtaes are suggested for a adult man daily"
    ],
    'completion': [
        "The perfect time to have breakfast is between 7 AM to 9 AM.",
        "Eating fruits in the morning on an empty stomach or mid-afternoon is ideal.",
        "It’s generally not advised to eat fruits at night as it may cause digestive issues.",
        "Ideal dinner time is between 6 PM and 8 PM, ideally 3 hours before bed.",
        "Drinking around 2 to 3 liters of water daily is recommended."
    ]
}

health_dataset = Dataset.from_dict(data)
health_dataset = health_dataset.train_test_split(test_size=0.2)


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 needs padding token
model = AutoModelForCausalLM.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
def preprocess_function(examples):
    inputs = [f"Question: {q}\nAnswer: {a}" for q,a in zip(examples["prompt"], examples["completion"])]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

tokenized_dataset = health_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [7]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./health_model_results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=2,
    logging_steps=5,
    report_to="wandb",
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,No log,4.842736
2,No log,2.953252
3,5.286000,2.054821
4,5.286000,1.636003
5,1.529500,1.489678
6,1.529500,1.445379
7,1.529500,1.432356
8,0.800300,1.425969
9,0.800300,1.421222
10,0.739000,1.418714


TrainOutput(global_step=20, training_loss=2.0886879444122313, metrics={'train_runtime': 238.0047, 'train_samples_per_second': 0.168, 'train_steps_per_second': 0.084, 'total_flos': 2612920320000.0, 'train_loss': 2.0886879444122313, 'epoch': 10.0})

In [9]:
trainer.save_model("./my_health_guru_model")
tokenizer.save_pretrained("./my_health_guru_model")


('./my_health_guru_model/tokenizer_config.json',
 './my_health_guru_model/special_tokens_map.json',
 './my_health_guru_model/vocab.json',
 './my_health_guru_model/merges.txt',
 './my_health_guru_model/added_tokens.json',
 './my_health_guru_model/tokenizer.json')

In [14]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [15]:
trainer.save_model('/content/drive/MyDrive/Finetuning_stuff/my_health_guru_model')
tokenizer.save_pretrained('/content/drive/MyDrive/Finetuning_stuff/my_health_guru_model')


('/content/drive/MyDrive/Finetuning_stuff/my_health_guru_model/tokenizer_config.json',
 '/content/drive/MyDrive/Finetuning_stuff/my_health_guru_model/special_tokens_map.json',
 '/content/drive/MyDrive/Finetuning_stuff/my_health_guru_model/vocab.json',
 '/content/drive/MyDrive/Finetuning_stuff/my_health_guru_model/merges.txt',
 '/content/drive/MyDrive/Finetuning_stuff/my_health_guru_model/added_tokens.json',
 '/content/drive/MyDrive/Finetuning_stuff/my_health_guru_model/tokenizer.json')

In [17]:
import os
from huggingface_hub import notebook_login

# Retrieve the API key from environment variables
api_key = os.environ.get("Huggingface_Api_Key")



In [19]:
model.push_to_hub("SaiSuraj99/my_health_guru_model")
tokenizer.push_to_hub("SaiSuraj99/my_health_guru_model")


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SaiSuraj99/my_health_guru_model/commit/1c582a10e0f92cb6d148e774df0e9240f7f288ab', commit_message='Upload tokenizer', commit_description='', oid='1c582a10e0f92cb6d148e774df0e9240f7f288ab', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SaiSuraj99/my_health_guru_model', endpoint='https://huggingface.co', repo_type='model', repo_id='SaiSuraj99/my_health_guru_model'), pr_revision=None, pr_num=None)

In [28]:
from transformers import pipeline

health_bot = pipeline("text-generation", model="./my_health_guru_model", tokenizer=tokenizer)

prompt = "Question: When is the best time to eat fruits\nAnswer:"
output = health_bot(prompt, max_length=100)
print(output[0]["generated_text"])


Device set to use cpu


Question: When is the best time to eat fruits
Answer: As soon as you eat fruits and vegetables.


In [29]:
health_bot(prompt, max_length=100, temperature=0.1, top_p=0.9, do_sample=True)


[{'generated_text': 'Question: When is the best time to eat fruits\nAnswer: The best time to eat fruits is when you are ready to eat.'}]

In [38]:
from transformers import pipeline

health_bot = pipeline("text-generation", model="./my_health_guru_model", tokenizer=tokenizer)

prompt = "Question: When is the best time to eat a banana\nAnswer:"
output = health_bot(prompt, max_length=100)
print(output[0]["generated_text"])


Device set to use cpu


Question: When is the best time to eat a banana
Answer: No, not when you're out of the food supply and want to eat them until you are ready
